In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import sys

if 'google.colab' in sys.modules:
  %tensorflow_version 2.x
import tensorflow as tf

tf.random.set_seed(1234)
AUTO = tf.data.experimental.AUTOTUNE
#import tensorflow_datasets as tfds

import os
import re
import numpy as np
from time import time
import matplotlib.pyplot as plt


import tensorflow.compat.v1 as tf

import time

print("Tensorflow version {}".format(tf.__version__))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Tensorflow version 2.15.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU {}'.format(tpu.cluster_spec().as_dict()['worker']))
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: {}".format(strategy.num_replicas_in_sync))

REPLICAS: 1


In [4]:
# Maximum sentence length
MAX_LENGTH = 40

# Maximum number of samples to preprocess
MAX_SAMPLES = 2000

# For tf.data.Dataset
BATCH_SIZE = 64 * strategy.num_replicas_in_sync
BUFFER_SIZE = 20000

# For Transformer
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1

EPOCHS = 120

In [5]:
import pandas as pd
df = pd.read_csv("/content/sample_data/BengaliEmpatheticConversationsCorpus .csv", encoding='utf-8')









In [6]:
df.head()

,Topics,Question-Title,Questions,Answers
0,পারিবারিক দ্বন্দ্ব,মা ও স্ত্রীর মধ্যে মতানৈক্য বৃদ্ধি,আমার স্ত্রী এবং মায়ের মধ্যে টানটান মতবিরোধ চ...,"আপনি যা বর্ণনা করছেন তাকে মনোবিজ্ঞানীরা ""ত্রি..."
1,"পদার্থের অপব্যবহার, আসক্তি",আমি ধূমপানে আসক্ত। আমি কিভাবে থামাতে পারি?,"আমি বাচ্চা নেওয়ার পরিকল্পনা করছি, তাই আমাকে ...",হাই। আপনার শিশুর (এবং নিজের) জন্য যা স্বাস্থ্...
2,পারিবারিক দ্বন্দ্ব,আমার পরিবারের কাছ থেকে গোপন রাখা,"আমার মনের মধ্যে গোপন আছে, এবং আমি জানি না তাদ...",মনে হচ্ছে গোপন রাখা এখন আপনার জন্য একটি সমস্য...
3,"আচরণগত পরিবর্তন, সামাজিক সম্পর্ক",অধিকারী হওয়ার অন্তর্নিহিত কারণ,আমি আমার সম্পর্কের ক্ষেত্রে অত্যন্ত অধিকারসূচক...,হ্যালো। এটা দুর্দান্ত যে আপনি উপলব্ধি করতে সক...
4,দুশ্চিন্তা,আমি কি ওষুধ ছাড়া উদ্বেগ নিয়ন্ত্রণ করতে পারি?,কয়েক বছর আগে আমার মাথায় আঘাত লেগেছিল এবং আমা...,আপনি বলেননি কি বা কত ওষুধ আপনি চেষ্টা করেছেন।...


In [7]:
df=df.drop(["Topics","Question-Title"],axis=1)
df.rename(columns={'Questions': 'Question', 'Answers': 'Answer'}, inplace=True)
# df = df.iloc[:15000]

In [8]:
df

,Question,Answer
0,আমার স্ত্রী এবং মায়ের মধ্যে টানটান মতবিরোধ চ...,"আপনি যা বর্ণনা করছেন তাকে মনোবিজ্ঞানীরা ""ত্রি..."
1,"আমি বাচ্চা নেওয়ার পরিকল্পনা করছি, তাই আমাকে ...",হাই। আপনার শিশুর (এবং নিজের) জন্য যা স্বাস্থ্...
2,"আমার মনের মধ্যে গোপন আছে, এবং আমি জানি না তাদ...",মনে হচ্ছে গোপন রাখা এখন আপনার জন্য একটি সমস্য...
3,আমি আমার সম্পর্কের ক্ষেত্রে অত্যন্ত অধিকারসূচক...,হ্যালো। এটা দুর্দান্ত যে আপনি উপলব্ধি করতে সক...
4,কয়েক বছর আগে আমার মাথায় আঘাত লেগেছিল এবং আমা...,আপনি বলেননি কি বা কত ওষুধ আপনি চেষ্টা করেছেন।...
...,...,...
5486,হ্যাঁ এটা সত্যিই,এটি একটি অবিচলিত গতিতে অর্জন করা হচ্ছে... এটি ...
5487,আমি কাজ শেষে আজ রেজিস্ট্রেশন সেন্টারে গিয়েছিল...,এটাই সমস্যা। তুমি সেখানে কি করছ?
5488,আমি কিছু বন্ধুদের সাথে ভলিবল খেলি। কাজের পরে আ...,চমৎকার আহ! তোমার জন্য ভালো. সম্ভবত একটি রান জন...
5489,আমি খুব রাগান্বিত হয়েছিলাম যখন আমার মা..যিনি ...,কি দারুন. তোর মাকে নিয়ে তোর খালা কি রেগে আছে?


In [9]:
# questions=[]
# answers=[]

# for i in df.Question:
#   questions.append(i)

# for i in df.Answer:
#   answers.append(i)

questions_eva=[]
answers_eva=[]

for i in df.Question:
  questions_eva.append(i)

for i in df.Answer:
  answers_eva.append(i)


# Remove rows with nan values in both 'Question' and 'Answer' columns
df_cleaned = df.dropna(subset=['Question', 'Answer'])

# Get the cleaned questions and answers
questions = df_cleaned['Question'].tolist()
answers = df_cleaned['Answer'].tolist()

In [10]:
print('Sample question: {}'.format(questions[17]))
print('Sample answer: {}'.format(answers[17]))
X = questions_eva
Y = answers_eva

Sample question: এখন আমার রাগ এবং বিশ্বাসের সমস্যা আছে। আমি কিভাবে এটি চিকিত্সা করতে পারি এবং নিজেকে ঠিক করতে পারি?
Sample answer:  আমি ধরে নেব যে কোন কারণেই হোক, আপনি হয় এই বিষয়ে কাউন্সেলিং নিতে চান না বা করতে পারছেন না। যাইহোক, আমি আপনাকে এটি করার জন্য অত্যন্ত অনুরোধ করছি কারণ দীর্ঘমেয়াদী শৈশব অপব্যবহার একজন প্রাপ্তবয়স্ক হিসাবে আপনার জীবনের জন্য নেতিবাচক পরিণতি ডেকে আনে, যেমনটি আপনি ইতিমধ্যেই সচেতন বলে মনে হচ্ছে৷ আপনি যৌন নিপীড়ন সঙ্গে যে চুক্তি বলতে পারেন. কিছু/বেশিরভাগ স্থানীয় এলাকায় বিনামূল্যে কাউন্সেলিং পরিষেবাও অফার করে। এটি দেখতে শুরু করার জন্য একটি ভাল জায়গা হবে, বিশেষ করে যদি কাউন্সেলিং খরচ একটি সমস্যা হয়।  জানা গুরুত্বপূর্ণ বিষয় হল যা ঘটেছে তা আপনার দোষ ছিল না এবং আপনি একটি সুখী জীবনযাপনের যোগ্য অতীত. এই জাতীয় কিছুর পরে রাগ এবং বিশ্বাসের সমস্যা থাকা আপনার সাথে যা ঘটেছে তার একটি সম্পূর্ণ স্বাভাবিক প্রতিক্রিয়া। রাগান্বিত? হ্যাঁ! তোমার ভাই যা করেছে তার জন্য, তোমাকে রক্ষা না করার জন্য তোমার বাবা-মায়ের কাছে। বিশ্বাস করতে সমস্যা হচ্ছে? অবশ্যই! যে লোকেরা আপনাকে ভালবাসতে

In [ ]:
pip install --upgrade pip


# Transformer

In [26]:
print(tf.__version__)
print(py)

2.14.0


In [ ]:
!pip install tensorflow-text





In [34]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
!pip install tensorflow_datasets

In [ ]:
import tensorflow_text as text
import tensorflow_datasets as tfds

# tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
#     questions + answers, target_vocab_size=2**13)
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)
# Define start and end token to indicate the start and end of a sentence
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# Vocabulary size plus start and end token
VOCAB_SIZE = tokenizer.vocab_size + 2

In [ ]:
print('Tokenized sample question: {}'.format(tokenizer.encode(questions[20])))

In [ ]:
# Tokenize, filter and pad sentences
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # tokenize sentence
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
    # check tokenized sentence max length
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)

  # pad tokenized sentences
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs


questions, answers = tokenize_and_filter(questions, answers)

In [ ]:
print('Vocab size: {}'.format(VOCAB_SIZE))
print('Number of samples: {}'.format(len(questions)))

In [ ]:
# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
print(dataset)

In [ ]:
def scaled_dot_product_attention(query, key, value, mask):
  """Calculate the attention weights. """
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # scale matmul_qk
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask to zero out padding tokens
  if mask is not None:
    logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k)
  attention_weights = tf.nn.softmax(logits, axis=-1)

  output = tf.matmul(attention_weights, value)

  return output

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # scaled dot-product attention
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # concatenation of heads
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # final linear layer
    outputs = self.dense(concat_attention)

    return outputs

In [ ]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [ ]:
print(create_padding_mask(tf.constant([[1, 2, 0, 3, 0], [0, 0, 0, 4, 5]])))

In [ ]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

In [ ]:

print(create_look_ahead_mask(tf.constant([[1, 2, 0, 4, 5]])))

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [ ]:
sample_pos_encoding = PositionalEncoding(50, 512)

plt.pcolormesh(sample_pos_encoding.pos_encoding.numpy()[0], cmap='RdBu')
plt.xlabel('Depth')
plt.xlim((0, 512))
plt.ylabel('Position')
plt.colorbar()
plt.show()

In [ ]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
sample_encoder_layer = encoder_layer(
    units=512,
    d_model=128,
    num_heads=4,
    dropout=0.3,
    name="sample_encoder_layer")
tf.keras.utils.plot_model(
    sample_encoder_layer, to_file='encoder_layer.png', show_shapes=True)

In [ ]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:

sample_encoder = encoder(
    vocab_size=2238,
    num_layers=2,
    units=512,
    d_model=128,
    num_heads=4,
    dropout=0.3,
    name="sample_encoder")

tf.keras.utils.plot_model(
   sample_encoder, to_file='encoder.png', show_shapes=True)

In [ ]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
sample_decoder_layer = decoder_layer(
    units=512,
    d_model=128,
    num_heads=4,
    dropout=0.3,
    name="sample_decoder_layer")

tf.keras.utils.plot_model(
    sample_decoder_layer, to_file='decoder_layer.png', show_shapes=True)

In [ ]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
sample_decoder = decoder(
    vocab_size=2238,
    num_layers=2,
    units=512,
    d_model=128,
    num_heads=4,
    dropout=0.3,
    name="sample_decoder")

tf.keras.utils.plot_model(
    sample_decoder, to_file='decoder.png', show_shapes=True)

In [ ]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
  # mask the future tokens for decoder inputs at the 1st attention block
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
  # mask the encoder outputs for the 2nd attention block
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [ ]:
sample_transformer = transformer(
    vocab_size=2238,
    num_layers=4,
    units=512,
    d_model=128,
    num_heads=4,
    dropout=0.3,
    name="sample_transformer")

tf.keras.utils.plot_model(
    sample_transformer, to_file='transformer.png', show_shapes=True)

In [ ]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [ ]:

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
sample_learning_rate = CustomSchedule(d_model=128)

plt.plot(sample_learning_rate(tf.range(200000, dtype=tf.float32)))
plt.ylabel("Learning Rate")
plt.xlabel("Train Step")

In [ ]:

# clear backend
tf.keras.backend.clear_session()

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

# initialize and compile model within strategy scope
with strategy.scope():
  model = transformer(
      vocab_size=VOCAB_SIZE,
      num_layers=NUM_LAYERS,
      units=UNITS,
      d_model=D_MODEL,
      num_heads=NUM_HEADS,
      dropout=DROPOUT)

  model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

model.summary()

In [ ]:
#model fit and train
history=model.fit(dataset, epochs=EPOCHS)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
def evaluate(sentence):

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # return the result if the predicted_id is equal to the end token
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # concatenated the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)


def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
output = predict('এখন আমার রাগ এবং বিশ্বাসের সমস্যা আছে। আমি কিভাবে এটি চিকিত্সা করতে পারি এবং নিজেকে ঠিক করতে পারি?')

In [ ]:
sentence="এখন আমার রাগ এবং বিশ্বাসের সমস্যা আছে। আমি কিভাবে এটি চিকিত্সা করতে পারি এবং নিজেকে ঠিক করতে পারি?"
for _ in range(10):
  sentence = predict(sentence)
  print('')

In [ ]:
print('Sample question: {}'.format(questions[17]))
print('Sample answer: {}'.format(answers[17]))

In [ ]:
print('Sample question: {}'.format(questions[17]))
print('Sample answer: {}'.format(answers[17]))

In [ ]:
import tqdm
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
X = questions_eva
Y = answers_eva

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
len (Y_test)

In [ ]:

# fr_preds = []
# for sentence in tqdm.tqdm(X_test):
#     fr_pred = predict(sentence)
#     fr_preds.append(fr_pred)
fr_preds = []
for sentence in tqdm.tqdm(X_test[10]):
    fr_pred = predict(sentence)
    fr_preds.append(fr_pred)

In [ ]:
references = Y_test

In [ ]:
bleu_score_1 = []
bleu_score_2 = []
bleu_score_3 = []
bleu_score_4 = []

for i in tqdm.tqdm(range(len(fr_preds))):
    pred = fr_preds[i].replace("<EOS>", "").replace("<PAD>", "").replace("<UNK>", "").replace("<GO>", "").rstrip()
    reference = references[i].lower().replace("<EOS>", "").replace("<PAD>", "").replace("<UNK>", "").replace("<GO>", "").rstrip()

    score_1 = sentence_bleu([reference.split()], pred.split(), weights=(1, 0, 0, 0))
    score_2 = sentence_bleu([reference.split()], pred.split(), weights=(0, 1, 0, 0))
    score_3 = sentence_bleu([reference.split()], pred.split(), weights=(0, 0, 1, 0))
    score_4 = sentence_bleu([reference.split()], pred.split(), weights=(0, 0, 0, 1))


#     score_1 = sentence_bleu([reference.split()], pred.split(), weights=(1, 0, 0, 0))
#     score_2 = sentence_bleu([reference.split()], pred.split(), weights=(0.5, 0.5, 0, 0))
#     score_3 = sentence_bleu([reference.split()], pred.split(), weights=(0.34, 0.33, 0.33, 0.))
#     score_4 = sentence_bleu([reference.split()], pred.split(), weights=(0.25, 0.25, 0.25, 0.25))



    bleu_score_1.append(score_1)
    bleu_score_2.append(score_2)
    bleu_score_3.append(score_3)
    bleu_score_4.append(score_4)

In [ ]:
print("The BLEU score individual 1-gram on our corpus is about {}".format(sum(bleu_score_1) / len(bleu_score_1)))
print("The BLEU score individual 2-gram on our corpus is about {}".format(sum(bleu_score_2) / len(bleu_score_2)))
print("The BLEU score individual 3-gram on our corpus is about {}".format(sum(bleu_score_3) / len(bleu_score_3)))
print("The BLEU score individual 4-gram on our corpus is about {}".format(sum(bleu_score_4) / len(bleu_score_4)))

In [ ]:
bleu_score_1 = []
bleu_score_2 = []
bleu_score_3 = []
bleu_score_4 = []

for i in tqdm.tqdm(range(len(fr_preds))):
    pred = fr_preds[i].replace("<EOS>", "").replace("<PAD>", "").replace("<UNK>", "").replace("<GO>", "").rstrip()
    reference = references[i].lower()
#     score_1 = sentence_bleu([reference.split()], pred.split(), weights=(1, 0, 0, 0))
#     score_2 = sentence_bleu([reference.split()], pred.split(), weights=(0, 1, 0, 0))
#     score_3 = sentence_bleu([reference.split()], pred.split(), weights=(0, 0, 1, 0))
#     score_4 = sentence_bleu([reference.split()], pred.split(), weights=(0, 0, 0, 1))


    score_1 = sentence_bleu([reference.split()], pred.split(), weights=(1, 0, 0, 0))
    score_2 = sentence_bleu([reference.split()], pred.split(), weights=(0.5, 0.5, 0, 0))
    score_3 = sentence_bleu([reference.split()], pred.split(), weights=(0.34, 0.33, 0.33, 0.))
    score_4 = sentence_bleu([reference.split()], pred.split(), weights=(0.25, 0.25, 0.25, 0.25))



    bleu_score_1.append(score_1)
    bleu_score_2.append(score_2)
    bleu_score_3.append(score_3)
    bleu_score_4.append(score_4)

In [ ]:
print("The BLEU score cumulative 1-gram on our corpus is about {}".format(sum(bleu_score_1) / len(bleu_score_1)))
print("The BLEU score cumulative 2-gram on our corpus is about {}".format(sum(bleu_score_2) / len(bleu_score_2)))
print("The BLEU score cumulative 3-gram on our corpus is about {}".format(sum(bleu_score_3) / len(bleu_score_3)))
print("The BLEU score cumulative 4-gram on our corpus is about {}".format(sum(bleu_score_4) / len(bleu_score_4)))

# Seq2Seq

In [25]:
from __future__ import print_function

import sys
import os
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt')
from keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense, Bidirectional
from keras.models import Model, load_model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [26]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = " ".join(text.split())
    return text

In [27]:
# Clean the data
clean_questions = []
for question in df.Question:
    clean_questions.append(clean_text(str(question)))
clean_answers = []
for answer in df.Answer:
    clean_answers.append(clean_text(str(answer)))

In [28]:
# Find the length of sentences (not using nltk due to processing speed)
lengths = []
# lengths.append([len(nltk.word_tokenize(sent)) for sent in clean_questions]) #nltk approach
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))
# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))

47.0
73.0
110.0
178.0


In [29]:
# Remove questions and answers that are shorter than 1 word and longer than 20 words.
min_line_length = 0
max_line_length = 15

# Filter out the questions that are too short/long
short_questions_temp = []
short_answers_temp = []

for i, question in enumerate(clean_questions):
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])

# Filter out the answers that are too short/long
short_questions = []
short_answers = []

for i, answer in enumerate(short_answers_temp):
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])

print(len(short_questions))
print(len(short_answers))

2106
2106


In [30]:
r = np.random.randint(1,len(short_questions))

for i in range(r, r+3):
    print(short_questions[i])
    print(short_answers[i])
    print()

এটি আসলে একটি নির্দিষ্ট সংস্করণ এটি পুরানো গেম থেকে সবকিছু আছে
ওহ বাহ আমি সেগুলি সব খেলিনি তবে আমি প্রথমটি করেছি।

আমি নতুন সুপার স্ম্যাশ ব্রোস গেমের জন্য উত্তেজিত যেটি শীঘ্রই আসছে
আমি এর সাথে পরিচিত নই আমাকে আরও বলুন।

এটি মারিও জেল্ডা এবং পোকেমন চরিত্রের মতো নিন্টেন্ডো চরিত্রগুলির সাথে একটি লড়াইয়ের খেলা
এটা মজার শোনাচ্ছে কোন গেম প্ল্যাটফর্ম



In [31]:
#choosing number of samples
num_samples = 15000  # Number of samples to train on.
short_questions = short_questions[:num_samples]
short_answers = short_answers[:num_samples]
#tokenizing the qns and answers
short_questions_tok = [nltk.word_tokenize(sent) for sent in short_questions]
short_answers_tok = [nltk.word_tokenize(sent) for sent in short_answers]

In [32]:
#train-validation split
data_size = len(short_questions_tok)

# We will use the first 0-80th %-tile (80%) of data for the training
training_input  = short_questions_tok[:round(data_size*(80/100))]
training_input  = [tr_input[::-1] for tr_input in training_input] #reverseing input seq for better performance
training_output = short_answers_tok[:round(data_size*(80/100))]

# We will use the remaining for validation
validation_input = short_questions_tok[round(data_size*(80/100)):]
validation_input  = [val_input[::-1] for val_input in validation_input] #reverseing input seq for better performance
validation_output = short_answers_tok[round(data_size*(80/100)):]

print('training size', len(training_input))
print('validation size', len(validation_input))

training size 1685
validation size 421


In [33]:
# Create a dictionary for the frequency of the vocabulary
# Create
vocab = {}
for question in short_questions_tok:
    for word in question:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

for answer in short_answers_tok:
    for word in answer:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [34]:
threshold = 2
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [35]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 5163
Size of vocab we will use: 2366


In [36]:
#we will create dictionaries to provide a unique integer for each word.
WORD_CODE_START = 1
WORD_CODE_PADDING = 0


word_num  = 2 #number 1 is left for WORD_CODE_START for model decoder later
encoding = {}
decoding = {1: 'START'}
for word, count in vocab.items():
    if count >= threshold: #get vocabularies that appear above threshold count
        encoding[word] = word_num
        decoding[word_num ] = word
        word_num += 1

print("No. of vocab used:", word_num)

No. of vocab used: 2368


In [37]:
#include unknown token for words not in dictionary
decoding[len(encoding)+2] = '<UNK>'
encoding['<UNK>'] = len(encoding)+2

In [38]:
dict_size = word_num+1
dict_size

2369

In [39]:
def transform(encoding, data, vector_size=20):

    transformed_data = np.zeros(shape=(len(data), vector_size))
    for i in range(len(data)):
        for j in range(min(len(data[i]), vector_size)):
            try:
                transformed_data[i][j] = encoding[data[i][j]]
            except:
                transformed_data[i][j] = encoding['<UNK>']
    return transformed_data

In [40]:
#encoding training set
INPUT_LENGTH = 15
OUTPUT_LENGTH = 5

encoded_training_input = transform(
    encoding, training_input, vector_size=INPUT_LENGTH)
encoded_training_output = transform(
    encoding, training_output, vector_size=OUTPUT_LENGTH)

print('encoded_training_input', encoded_training_input.shape)
print('encoded_training_output', encoded_training_output.shape)

encoded_training_input (1685, 15)
encoded_training_output (1685, 5)


In [41]:
#encoding validation set
encoded_validation_input = transform(
    encoding, validation_input, vector_size=INPUT_LENGTH)
encoded_validation_output = transform(
    encoding, validation_output, vector_size=OUTPUT_LENGTH)

print('encoded_validation_input', encoded_validation_input.shape)
print('encoded_validation_output', encoded_validation_output.shape)

encoded_validation_input (421, 15)
encoded_validation_output (421, 5)


In [42]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [43]:
INPUT_LENGTH = 15
OUTPUT_LENGTH = 5

encoder_input = Input(shape=(INPUT_LENGTH,))
decoder_input = Input(shape=(OUTPUT_LENGTH,))

In [44]:
from keras.layers import SimpleRNN

encoder = Embedding(dict_size, 128, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
encoder = LSTM(512, return_sequences=True, unroll=True)(encoder)
encoder_last = encoder[:,-1,:]

print('encoder', encoder)
print('encoder_last', encoder_last)

decoder = Embedding(dict_size, 128, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)
decoder = LSTM(512, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)

encoder KerasTensor(type_spec=TensorSpec(shape=(None, 15, 512), dtype=tf.float32, name=None), name='lstm/transpose_2:0', description="created by layer 'lstm'")
encoder_last KerasTensor(type_spec=TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), name='tf.__operators__.getitem/strided_slice:0', description="created by layer 'tf.__operators__.getitem'")
decoder KerasTensor(type_spec=TensorSpec(shape=(None, 5, 512), dtype=tf.float32, name=None), name='lstm_1/transpose_2:0', description="created by layer 'lstm_1'")


In [45]:
from keras.layers import Activation, dot, concatenate

# Equation (7) with 'dot' score from Section 3.1 in the paper.
# Note that we reuse Softmax-activation layer instead of writing tensor calculation
attention = dot([decoder, encoder], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

# Has another weight + tanh layer as described in equation (5) of the paper
output = TimeDistributed(Dense(512, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(dict_size, activation="softmax"))(output)
print('output', output)

attention KerasTensor(type_spec=TensorSpec(shape=(None, 5, 15), dtype=tf.float32, name=None), name='attention/Softmax:0', description="created by layer 'attention'")
context KerasTensor(type_spec=TensorSpec(shape=(None, 5, 512), dtype=tf.float32, name=None), name='dot_1/MatMul:0', description="created by layer 'dot_1'")
decoder_combined_context KerasTensor(type_spec=TensorSpec(shape=(None, 5, 1024), dtype=tf.float32, name=None), name='concatenate/concat:0', description="created by layer 'concatenate'")
output KerasTensor(type_spec=TensorSpec(shape=(None, 5, 2369), dtype=tf.float32, name=None), name='time_distributed_1/Reshape_1:0', description="created by layer 'time_distributed_1'")


In [46]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['categorical_accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 15)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 15, 128)              303232    ['input_1[0][0]']             
                                                                                                  
 input_2 (InputLayer)        [(None, 5)]                  0         []                            
                                                                                                  
 lstm (LSTM)                 (None, 15, 512)              1312768   ['embedding[0][0]']           
                                                                                              

In [47]:
training_encoder_input = encoded_training_input
training_decoder_input = np.zeros_like(encoded_training_output)
training_decoder_input[:, 1:] = encoded_training_output[:,:-1]
training_decoder_input[:, 0] = WORD_CODE_START
training_decoder_output =np.eye(dict_size)[encoded_training_output.astype('int')]

validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)
validation_decoder_input[:, 1:] = encoded_validation_output[:,:-1]
validation_decoder_input[:, 0] = WORD_CODE_START
validation_decoder_output = np.eye(dict_size)[encoded_validation_output.astype('int')]

In [ ]:
model.fit(x=[training_encoder_input, training_decoder_input], y=[training_decoder_output],
          validation_data=([validation_encoder_input, validation_decoder_input], [validation_decoder_output]),
          #validation_split=0.05,
          batch_size=64, epochs=100)

In [ ]:
def prediction(raw_input):
    clean_input = clean_text(raw_input)
    input_tok = [nltk.word_tokenize(clean_input)]
    input_tok = [input_tok[0][::-1]]  #reverseing input seq
    encoder_input = transform(encoding, input_tok, 15)
    decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_LENGTH))
    decoder_input[:,0] = WORD_CODE_START
    for i in range(1, OUTPUT_LENGTH):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return output

def decode(decoding, vector):

    text = ''
    for i in vector:
        if i == 0:
            break
        text += ' '
        text += decoding[i]
    return text

In [ ]:
for i in range(15):
    seq_index = np.random.randint(1, len(short_questions))
    output = prediction(short_questions[seq_index])
    print ('Q:', short_questions[seq_index])
    print ('A:', decode(decoding, output[0]))

In [ ]:
import tqdm
from nltk.translate.bleu_score import sentence_bleu

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
len (Y_test)

In [ ]:
len(X_test)
X_test=X_test[0:10]

In [ ]:
fr_preds = []
for sentence in tqdm.tqdm(X_test):
    fr_pred = decode(decoding, prediction(sentence)[0])
    fr_preds.append(fr_pred)

In [ ]:

references = Y_test

In [ ]:
bleu_score_1 = []
bleu_score_2 = []
bleu_score_3 = []
bleu_score_4 = []

for i in tqdm.tqdm(range(len(fr_preds))):

    pred = fr_preds[i].replace("<EOS>", "").replace("<PAD>", "").replace("<UNK>", "").replace("<GO>", "").rstrip()
    reference = references[i].lower().replace("<EOS>", "").replace("<PAD>", "").replace("<UNK>", "").replace("<GO>", "").rstrip()

    score_1 = sentence_bleu([reference.split()], pred.split(), weights=(1, 0, 0, 0))
    score_2 = sentence_bleu([reference.split()], pred.split(), weights=(0, 1, 0, 0))
    score_3 = sentence_bleu([reference.split()], pred.split(), weights=(0, 0, 1, 0))
    score_4 = sentence_bleu([reference.split()], pred.split(), weights=(0, 0, 0, 1))


#     score_1 = sentence_bleu([reference.split()], pred.split(), weights=(1, 0, 0, 0))
#     score_2 = sentence_bleu([reference.split()], pred.split(), weights=(0.5, 0.5, 0, 0))
#     score_3 = sentence_bleu([reference.split()], pred.split(), weights=(0.34, 0.33, 0.33, 0.))
#     score_4 = sentence_bleu([reference.split()], pred.split(), weights=(0.25, 0.25, 0.25, 0.25))



    bleu_score_1.append(score_1)
    bleu_score_2.append(score_2)
    bleu_score_3.append(score_3)
    bleu_score_4.append(score_4)

In [ ]:

print("The BLEU score individual 1-gram on our corpus is about {}".format(sum(bleu_score_1) / len(bleu_score_1)))
print("The BLEU score individual 2-gram on our corpus is about {}".format(sum(bleu_score_2) / len(bleu_score_2)))
print("The BLEU score individual 3-gram on our corpus is about {}".format(sum(bleu_score_3) / len(bleu_score_3)))
print("The BLEU score individual 4-gram on our corpus is about {}".format(sum(bleu_score_4) / len(bleu_score_4)))

In [ ]:
bleu_score_1 = []
bleu_score_2 = []
bleu_score_3 = []
bleu_score_4 = []

for i in tqdm.tqdm(range(len(fr_preds))):

    pred = fr_preds[i].replace("<EOS>", "").replace("<PAD>", "").replace("<UNK>", "").replace("<GO>", "").rstrip()
    reference = references[i].lower()
#     计算BLEU分数
#     score_1 = sentence_bleu([reference.split()], pred.split(), weights=(1, 0, 0, 0))
#     score_2 = sentence_bleu([reference.split()], pred.split(), weights=(0, 1, 0, 0))
#     score_3 = sentence_bleu([reference.split()], pred.split(), weights=(0, 0, 1, 0))
#     score_4 = sentence_bleu([reference.split()], pred.split(), weights=(0, 0, 0, 1))


    score_1 = sentence_bleu([reference.split()], pred.split(), weights=(1, 0, 0, 0))
    score_2 = sentence_bleu([reference.split()], pred.split(), weights=(0.5, 0.5, 0, 0))
    score_3 = sentence_bleu([reference.split()], pred.split(), weights=(0.34, 0.33, 0.33, 0.))
    score_4 = sentence_bleu([reference.split()], pred.split(), weights=(0.25, 0.25, 0.25, 0.25))



    bleu_score_1.append(score_1)
    bleu_score_2.append(score_2)
    bleu_score_3.append(score_3)
    bleu_score_4.append(score_4)

In [ ]:
print("The BLEU score cumulative 1-gram on our corpus is about {}".format(sum(bleu_score_1) / len(bleu_score_1)))
print("The BLEU score cumulative 2-gram on our corpus is about {}".format(sum(bleu_score_2) / len(bleu_score_2)))
print("The BLEU score cumulative 3-gram on our corpus is about {}".format(sum(bleu_score_3) / len(bleu_score_3)))
print("The BLEU score cumulative 4-gram on our corpus is about {}".format(sum(bleu_score_4) / len(bleu_score_4)))